In [1]:
import pandas as pd

# Carga del dataset
try:
    df = pd.read_csv('ds_salaries.csv')
    print("✅ Dataset cargado correctamente.")
except FileNotFoundError:
    print("❌ Error: No se encuentra el archivo. Verifica el nombre.")

# 1. Mirada rápida
print(f"\nDimensiones: {df.shape[0]} filas, {df.shape[1]} columnas")

# 2. ¿Qué columnas tenemos?
print("\nColumnas disponibles:")
print(df.columns.tolist())

# 3. ¿Qué tan diversos son los roles?
print(f"\nCantidad de Job Titles distintos: {df['job_title'].nunique()}")
print("Top 10 Roles más comunes:")
print(df['job_title'].value_counts().head(10))

# 4. Check de Nulos
print("\nNulos por columna:")
print(df.isnull().sum())

✅ Dataset cargado correctamente.

Dimensiones: 93597 filas, 11 columnas

Columnas disponibles:
['work_year', 'experience_level', 'employment_type', 'job_title', 'salary', 'salary_currency', 'salary_in_usd', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']

Cantidad de Job Titles distintos: 317
Top 10 Roles más comunes:
job_title
Data Scientist               13848
Data Engineer                11323
Software Engineer            10133
Data Analyst                  9081
Machine Learning Engineer     6643
Engineer                      5804
Manager                       3832
Research Scientist            2641
Analyst                       2314
Applied Scientist             1816
Name: count, dtype: int64

Nulos por columna:
work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_

In [2]:
import numpy as np

In [5]:
# ==========================================
# 1. FUNCIÓN DE CATEGORIZACIÓN (TAXONOMÍA)
# ==========================================
def categorize_job_title(title):
    title = title.lower()

    # Orden de prioridad: Liderazgo primero, luego especialidad
    if any(x in title for x in ['manager', 'head', 'director', 'lead', 'principal', 'vp', 'chief', 'exec']):
        return 'Leadership & Management'
    elif any(x in title for x in ['machine learning', 'ai ', 'artificial intelligence', 'nlp', 'computer vision', 'deep learning']):
        return 'AI & Machine Learning'
    elif any(x in title for x in ['scientist', 'research']):
        return 'Data Science'
    elif any(x in title for x in ['architect', 'engineer', 'etl', 'pipeline']):
        # Cuidado aquí: Software Engineer caerá aquí si no filtramos, pero vamos a dejarlo
        # o separarlo si queremos comparar Data Eng vs Software Eng.
        # Para simplificar, agrupemos Ingeniería de Datos.
        if 'software' in title:
            return 'Software & Tech'
        return 'Data Engineering'
    elif any(x in title for x in ['analyst', 'analytics', 'bi ', 'business intelligence']):
        return 'Data Analysis'
    else:
        return 'Other'

# Aplicamos la función
df['Job_Category'] = df['job_title'].apply(categorize_job_title)

In [6]:
# ==========================================
# 2. LIMPIEZA DE CÓDIGOS (UX)
# ==========================================
# Mapeo de Experiencia
experience_map = {
    'EN': 'Junior (Entry)',
    'MI': 'Mid-Level',
    'SE': 'Senior',
    'EX': 'Executive'
}
df['Experience_Clean'] = df['experience_level'].map(experience_map)

# Mapeo de Modalidad
remote_map = {
    0: 'On-Site',
    50: 'Hybrid',
    100: 'Remote'
}
df['Work_Model'] = df['remote_ratio'].map(remote_map)


In [9]:
# ==========================================
# 3. FILTROS DE CALIDAD
# ==========================================
df_final = df[df['employment_type'] == 'FT'].copy()

In [10]:
# ==========================================
# 4. VALIDACIÓN DE RESULTADOS
# ==========================================
print("Distribución de Categorías creadas:")
print(df_final['Job_Category'].value_counts())

print("\nEjemplo de datos transformados:")
print(df_final[['job_title', 'Job_Category', 'salary_in_usd', 'Experience_Clean']].head())

Distribución de Categorías creadas:
Job_Category
Data Engineering           24700
Data Science               20378
Data Analysis              14320
Software & Tech            10621
AI & Machine Learning       9541
Leadership & Management     8383
Other                       5147
Name: count, dtype: int64

Ejemplo de datos transformados:
            job_title           Job_Category  salary_in_usd Experience_Clean
0  Research Scientist           Data Science         208000        Mid-Level
1  Research Scientist           Data Science         147000        Mid-Level
2  Research Scientist           Data Science         173000           Senior
3  Research Scientist           Data Science         117000           Senior
4         AI Engineer  AI & Machine Learning         100000        Mid-Level


In [11]:
# ==========================================
# 5. EXPORTACIÓN
# ==========================================
df_final.to_csv('AI_Salaries_Cleaned.csv', index=False)
print("\nArchivo 'AI_Salaries_Cleaned.csv' exportado exitosamente.")


Archivo 'AI_Salaries_Cleaned.csv' exportado exitosamente.
